# Practica APA - predicció superfície cremada d'un incendi
Sergi Curto Panisello,
Joan Melchor Lladó

## Imports

In [36]:
import pandas as pd
import numpy as np
import tabulate
from geopy.geocoders import Nominatim

## Obtenim les dades
Procedim a agafar les dades en format csv descarregades de https://datos.civio.es/dataset/todos-los-incendios-forestales/

In [37]:
df = pd.read_csv('fires-all.csv').set_index('id')
print(df.describe())


                 id    superficie           lat           lng  \
count  8.264000e+04  82640.000000  82616.000000  82616.000000   
mean   2.007259e+09     19.888085     41.763721     -5.664360   
std    4.247106e+06    223.787536      2.019672      4.394031   
min    2.001010e+09      1.000000      0.490720  -1000.000000   
25%    2.003390e+09      1.500000     40.876544     -7.257298   
50%    2.006330e+09      3.000000     42.371123     -6.019478   
75%    2.011150e+09      7.150000     43.143145     -4.303078   
max    2.015510e+09  28879.100000     87.824157    242.755603   

       latlng_explicit   idcomunidad   idprovincia   idmunicipio  \
count     82640.000000  82640.000000  82640.000000  82640.000000   
mean          0.741626      7.850278     28.411375     77.795946   
std           0.437743      5.419922     11.727155     98.867769   
min           0.000000      1.000000      1.000000      1.000000   
25%           0.000000      3.000000     21.000000     27.000000   
50%   

In [52]:
geolocator = Nominatim(user_agent="apaProject")
location = geolocator.geocode('mont-ras')
print(location.address)
print((location.latitude, location.longitude))

Mont-ras, Baix Empordà, Girona, Catalunya, España
(41.9080196, 3.144796)
{'place_id': 256938123, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 345519, 'boundingbox': ['41.8670361', '41.9185439', '3.1235748', '3.1795064'], 'lat': '41.9080196', 'lon': '3.144796', 'display_name': 'Mont-ras, Baix Empordà, Girona, Catalunya, España', 'class': 'boundary', 'type': 'administrative', 'importance': 0.6785845118429703, 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_boundary_administrative.p.20.png'}


In [ ]:
# Si separem solament Catalunya
cat = df.loc[df['idcomunidad'] == 2]
print(cat.head)
print('Shape')
print(cat.shape)

print("ID especial")
print(pd.concat([df.loc[df['id'] == 2001320802], df.loc[df['id'] == 2002151000]]).to_markdown())
# print(df.loc[df['id'] == 2002151000])

print("ID cremat molt")
print(df.loc[df['superficie'] == 28879.10])

## Preprocessing

### Tractament de missing values

In [ ]:
print(df.duplicated(subset='id', keep="first"))

In [ ]:
# Columnes amb missing values, també es pot veure al describe del
# dataset a les variables que no tinguin un count de 82640
print(df.columns[df.isnull().any()].tolist())

# Sobre la "causa supuesta" és normal que hi hagin instàncies sense valor ja que 
# idmunicipio 999, i 998 ens carreguem els que no tinguin lat i long ja que són incendis originats fora d'Espanya
df.dropna(subset=['lat', 'lng'], inplace=True)

# Comencem per emplenar els missing values de muertos i heridos ja que
# si no hi han dades suposarem que són 0.
df['muertos'].fillna(0, inplace=True)
df['heridos'].fillna(0, inplace=True)

# Causa supuesta la posem a 0
df['causa_supuesta'].fillna(0, inplace=True)

# Posem a 0 gastos i perdidas
df['gastos'].fillna(0, inplace=True)
df['perdidas'].fillna(0, inplace=True)

# df.describe()

print(df.columns[df.isnull().any()].tolist())

In [ ]:
aux = df.loc[df['idmunicipio'] == 999]
latMissing = df.loc[np.isnan(df['lat'])]
print(aux.shape)
print(latMissing.shape)
print(latMissing)

## Separem target del dataset
Volem predir la superfície cremada donat un incendi per tant separem aquesta variable de la resta.

In [ ]:
X = df.drop('superficie', axis=1)
y = df['superficie']

# Comprovem que al target no hi hagin nulls
print(y.isnull().values.any())
